In [ ]:
%config InlineBackend.figure_formats = ['svg']
import os

STATIC_WEB_PAGE = {"EXECUTE_NB", "READTHEDOCS"}.intersection(os.environ)

```{autolink-concat}
```

````{margin}
```{spec} Square root over arrays with negative values
:id: TR-000
:status: Implemented
:tags: lambdification;sympy

This report has been implemented in {pr}`ComPWA/tensorwaves#284`.

```
````

# Riemann sheets

<!-- cspell:disable -->

In [ ]:
%pip install -q sympy==1.12

Note: you may need to restart the kernel to use updated packages.
